# Modelling of lyrics of various artists.

In [19]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.compose import make_column_transformer
from sklearn.ensemble import RandomForestClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import accuracy_score
from sklearn import preprocessing
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [4]:
df= pd.read_csv('./src/Songs2.csv')

In [5]:
df=df.drop('Unnamed: 0', axis=1)

In [6]:
df['Artist'].value_counts(dropna= False)

Bob Marley         108
Led Zeppelin        89
Michael Jackson     86
Name: Artist, dtype: int64

In [7]:
df.head()

,Artist,Lyrics
0,Bob Marley,"Stir it up, little darlin', stir it up, me on,..."
1,Bob Marley,"Huh, please don't you rock my boat\n'Cause I d..."
2,Bob Marley,Zion train is coming our way\nThe Zion train i...
3,Bob Marley,Woman hold her head and cry\n'Cause her son ha...
4,Bob Marley,"Stir it up little darling, stir it up\nCome on..."


In [8]:
y = df['Artist']
X= df['Lyrics'].reset_index()

In [9]:
X_train, X_test, y_train, y_test = train_test_split(X,y)

In [21]:
preprocess = make_column_transformer(
                (TfidfVectorizer(), 'Lyrics'))

model_pipe_LR = make_pipeline(
                            preprocess,
                            LogisticRegression()
)

model_pipe_RF = make_pipeline(
                            preprocess,
                            RandomForestClassifier()
)

model_pipe_NB = make_pipeline(
                            preprocess,
                            MultinomialNB()
)

model_pipe_SVM = make_pipeline(
                            preprocess,
                            SVC()
)


In [37]:
cv_svc = GridSearchCV(
                    estimator = model_pipe_SVM,
                    param_grid = grid_svc,
                    scoring = 'accuracy',
                    cv = 3, 
                    return_train_score=True )

In [36]:
grid_svc = [{'svc__kernel': ['rbf'], 
             'svc__gamma': [1e-3, 1e-4],
             'svc__C': [1, 10, 100, 1000]},
            {'svc__kernel': ['linear'], 'svc__C': [1, 10, 100, 1000]}]

In [38]:
cv_svc.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('tfidfvectorizer',
                                                                         TfidfVectorizer(),
                                                                         'Lyrics')])),
                                       ('svc', SVC())]),
             param_grid=[{'svc__C': [1, 10, 100, 1000],
                          'svc__gamma': [0.001, 0.0001],
                          'svc__kernel': ['rbf']},
                         {'svc__C': [1, 10, 100, 1000],
                          'svc__kernel': ['linear']}],
             return_train_score=True, scoring='accuracy')

In [41]:
cv_results=cv_results[['rank_test_score', 'mean_test_score', 'mean_train_score',
                      'param_svc__C',
                      'param_svc__gamma',
                      'params']]

In [39]:
cv_results = pd.DataFrame(cv.cv_results_)

In [42]:
cv_results

,rank_test_score,mean_test_score,mean_train_score,param_svc__C,param_svc__gamma,params
0,8,0.372636,0.372640,1,0.001,"{'svc__C': 1, 'svc__gamma': 0.001, 'svc__kerne..."
1,8,0.372636,0.372640,1,0.0001,"{'svc__C': 1, 'svc__gamma': 0.0001, 'svc__kern..."
2,8,0.372636,0.372640,10,0.001,"{'svc__C': 10, 'svc__gamma': 0.001, 'svc__kern..."
3,8,0.372636,0.372640,10,0.0001,"{'svc__C': 10, 'svc__gamma': 0.0001, 'svc__ker..."
4,6,0.391415,0.415193,100,0.001,"{'svc__C': 100, 'svc__gamma': 0.001, 'svc__ker..."
5,8,0.372636,0.372640,100,0.0001,"{'svc__C': 100, 'svc__gamma': 0.0001, 'svc__ke..."
6,1,0.745540,0.995289,1000,0.001,"{'svc__C': 1000, 'svc__gamma': 0.001, 'svc__ke..."
7,6,0.391415,0.415193,1000,0.0001,"{'svc__C': 1000, 'svc__gamma': 0.0001, 'svc__k..."
8,5,0.736083,0.995289,1,NaN,"{'svc__C': 1, 'svc__kernel': 'linear'}"
9,2,0.745473,1.000000,10,NaN,"{'svc__C': 10, 'svc__kernel': 'linear'}"


In [ ]:
#cross-validation

In [22]:
# at end ..after grid search cv

model_pipe_LR.fit(X_train,y_train)
model_pipe_RF.fit(X_train,y_train)
model_pipe_NB.fit(X_train,y_train)
model_pipe_SVM.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(),
                                                  'Lyrics')])),
                ('svc', SVC())])

In [27]:
# predict the data
y_train_pred_LR = model_pipe_LR.predict(X_train)
y_test_pred_LR = model_pipe_LR.predict(X_test)

y_train_pred_RF = model_pipe_RF.predict(X_train)
y_test_pred_RF = model_pipe_RF.predict(X_test)

y_train_pred_NB = model_pipe_NB.predict(X_train)
y_test_pred_NB = model_pipe_NB.predict(X_test)

y_train_pred_SVM = model_pipe_SVM.predict(X_train)
y_test_pred_SVM = model_pipe_SVM.predict(X_test)


In [28]:
accuracy_score(y_test_pred_LR, y_test)

0.7323943661971831

In [29]:
accuracy_score(y_test_pred_RF, y_test)

0.6619718309859155

In [30]:
accuracy_score(y_test_pred_NB, y_test)

0.5633802816901409

In [31]:
accuracy_score(y_test_pred_SVM, y_test)

0.7323943661971831

In [15]:
model_pipe_RF = make_pipeline(
            preprocess,
            RandomForestClassifier())

In [16]:
model_pipe_RF.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(),
                                                  'Lyrics')])),
                ('randomforestclassifier', RandomForestClassifier())])

In [17]:
# predict the data
y_train_pred_RF = model_pipe_RF.predict(X_train)
y_test_pred_RF = model_pipe_RF.predict(X_test)

In [18]:
accuracy_score(y_test_pred_RF, y_test)

0.647887323943662

In [32]:
model_pipe_SVM.get_params()

{'memory': None,
 'steps': [('columntransformer',
   ColumnTransformer(transformers=[('tfidfvectorizer', TfidfVectorizer(),
                                    'Lyrics')])),
  ('svc', SVC())],
 'verbose': False,
 'columntransformer': ColumnTransformer(transformers=[('tfidfvectorizer', TfidfVectorizer(),
                                  'Lyrics')]),
 'svc': SVC(),
 'columntransformer__n_jobs': None,
 'columntransformer__remainder': 'drop',
 'columntransformer__sparse_threshold': 0.3,
 'columntransformer__transformer_weights': None,
 'columntransformer__transformers': [('tfidfvectorizer',
   TfidfVectorizer(),
   'Lyrics')],
 'columntransformer__verbose': False,
 'columntransformer__tfidfvectorizer': TfidfVectorizer(),
 'columntransformer__tfidfvectorizer__analyzer': 'word',
 'columntransformer__tfidfvectorizer__binary': False,
 'columntransformer__tfidfvectorizer__decode_error': 'strict',
 'columntransformer__tfidfvectorizer__dtype': numpy.float64,
 'columntransformer__tfidfvectorizer_

In [45]:
model_pipe_SVM_mod = make_pipeline(
                            preprocess,
                            SVC(C = 1000, gamma = 0.001) 
)

In [46]:
model_pipe_SVM_mod.fit(X_train,y_train)

Pipeline(steps=[('columntransformer',
                 ColumnTransformer(transformers=[('tfidfvectorizer',
                                                  TfidfVectorizer(),
                                                  'Lyrics')])),
                ('svc', SVC(C=1000, gamma=0.001))])

In [47]:
y_train_pred_SVM_mod = model_pipe_SVM_mod.predict(X_train)
y_test_pred_SVM_mod = model_pipe_SVM_mod.predict(X_test)

In [48]:
accuracy_score(y_test_pred_SVM_mod, y_test)

0.7605633802816901

In [49]:
y_pred = cv.best_estimator_.predict(X)

In [50]:
y_pred

array(['Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Led Zeppelin',
       'Michael Jackson', 'Bob Marley', 'Michael Jackson', 'Bob Marley',
       'Michael Jackson', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bo

In [64]:
grid = {'columntransformer__tfidfvectorizer__lowercase': [True, False],
        'columntransformer__tfidfvectorizer__ngram_range': [(1, 1), (1,2)],
        'randomforestclassifier__max_depth': [2,3,4]
       }

In [67]:
cv = GridSearchCV(
                    estimator = model_pipe_RF,
                    param_grid = grid,
                    scoring = 'accuracy',
                    cv = 3, 
                    return_train_score=True )

In [17]:
X.columns

Index(['index', 'Lyrics'], dtype='object')

In [68]:
cv.fit(X_train, y_train)

GridSearchCV(cv=3,
             estimator=Pipeline(steps=[('columntransformer',
                                        ColumnTransformer(transformers=[('tfidfvectorizer',
                                                                         TfidfVectorizer(),
                                                                         'Lyrics')])),
                                       ('randomforestclassifier',
                                        RandomForestClassifier())]),
             param_grid={'columntransformer__tfidfvectorizer__lowercase': [True,
                                                                           False],
                         'columntransformer__tfidfvectorizer__ngram_range': [(1,
                                                                              1),
                                                                             (1,
                                                                              2)],
                       

In [69]:
cv_results = pd.DataFrame(cv.cv_results_)

In [70]:
cv_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_columntransformer__tfidfvectorizer__lowercase,param_columntransformer__tfidfvectorizer__ngram_range,param_randomforestclassifier__max_depth,params,split0_test_score,split1_test_score,split2_test_score,mean_test_score,std_test_score,rank_test_score,split0_train_score,split1_train_score,split2_train_score,mean_train_score,std_train_score
0,0.115634,0.005373,0.018910,0.000641,True,"(1, 1)",2,{'columntransformer__tfidfvectorizer__lowercas...,0.545455,0.522727,0.511364,0.526515,0.014173,6,0.721591,0.715909,0.750000,0.729167,0.014913
1,0.117442,0.000205,0.019392,0.000687,True,"(1, 1)",3,{'columntransformer__tfidfvectorizer__lowercas...,0.511364,0.556818,0.545455,0.537879,0.019314,4,0.880682,0.823864,0.795455,0.833333,0.035432
2,0.136099,0.027136,0.018799,0.000140,True,"(1, 1)",4,{'columntransformer__tfidfvectorizer__lowercas...,0.636364,0.522727,0.522727,0.560606,0.053569,1,0.926136,0.863636,0.897727,0.895833,0.025551
3,0.172403,0.014679,0.031465,0.000505,True,"(1, 2)",2,{'columntransformer__tfidfvectorizer__lowercas...,0.522727,0.454545,0.511364,0.496212,0.029826,10,0.693182,0.715909,0.698864,0.702652,0.009657
4,0.173208,0.015395,0.031485,0.000931,True,"(1, 2)",3,{'columntransformer__tfidfvectorizer__lowercas...,0.500000,0.477273,0.556818,0.511364,0.033454,8,0.840909,0.710227,0.772727,0.774621,0.053367
5,0.155835,0.000233,0.030034,0.000484,True,"(1, 2)",4,{'columntransformer__tfidfvectorizer__lowercas...,0.568182,0.545455,0.522727,0.545455,0.018557,3,0.886364,0.784091,0.818182,0.829545,0.042519
6,0.114097,0.001231,0.019061,0.000170,False,"(1, 1)",2,{'columntransformer__tfidfvectorizer__lowercas...,0.477273,0.488636,0.545455,0.503788,0.029826,9,0.778409,0.710227,0.715909,0.734848,0.030889
7,0.110833,0.000937,0.017880,0.000372,False,"(1, 1)",3,{'columntransformer__tfidfvectorizer__lowercas...,0.556818,0.511364,0.534091,0.534091,0.018557,5,0.886364,0.812500,0.818182,0.839015,0.033561
8,0.115793,0.002613,0.019309,0.000130,False,"(1, 1)",4,{'columntransformer__tfidfvectorizer__lowercas...,0.579545,0.522727,0.568182,0.556818,0.024548,2,0.971591,0.948864,0.903409,0.941288,0.028346
9,0.147964,0.002805,0.029889,0.000629,False,"(1, 2)",2,{'columntransformer__tfidfvectorizer__lowercas...,0.465909,0.443182,0.511364,0.473485,0.028346,12,0.676136,0.670455,0.704545,0.683712,0.014913


In [22]:
cv_results.columns

Index(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time',
       'param_columntransformer__tfidfvectorizer__lowercase',
       'param_columntransformer__tfidfvectorizer__ngram_range',
       'param_randomforestclassifier__max_depth', 'params',
       'split0_test_score', 'split1_test_score', 'split2_test_score',
       'mean_test_score', 'std_test_score', 'rank_test_score',
       'split0_train_score', 'split1_train_score', 'split2_train_score',
       'mean_train_score', 'std_train_score'],
      dtype='object')

In [74]:
cv_results=cv_results[['rank_test_score', 'mean_test_score', 'mean_train_score',
                      'param_columntransformer__tfidfvectorizer__lowercase',
                      'param_columntransformer__tfidfvectorizer__ngram_range',
                      'param_randomforestclassifier__max_depth']]

KeyError: "['param_randomforestclassifier__max_depth'] not in index"

In [75]:
cv_results

,rank_test_score,mean_test_score,mean_train_score,param_columntransformer__tfidfvectorizer__lowercase,param_columntransformer__tfidfvectorizer__ngram_range,param_columntransformer__tfidfvectorizer__ngram_range
0,6,0.526515,0.729167,True,"(1, 1)","(1, 1)"
1,4,0.537879,0.833333,True,"(1, 1)","(1, 1)"
2,1,0.560606,0.895833,True,"(1, 1)","(1, 1)"
3,10,0.496212,0.702652,True,"(1, 2)","(1, 2)"
4,8,0.511364,0.774621,True,"(1, 2)","(1, 2)"
5,3,0.545455,0.829545,True,"(1, 2)","(1, 2)"
6,9,0.503788,0.734848,False,"(1, 1)","(1, 1)"
7,5,0.534091,0.839015,False,"(1, 1)","(1, 1)"
8,2,0.556818,0.941288,False,"(1, 1)","(1, 1)"
9,12,0.473485,0.683712,False,"(1, 2)","(1, 2)"


In [36]:
y_pred = cv.best_estimator_.predict(X)

In [37]:
y_pred 

array(['Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob Marley',
       'Bob Marley', 'Bob Marley', 'Bob Marley', 'Bob M

In [38]:
accuracy_score(y_pred, y)

0.7443181818181818